# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Наша задача - проанализировать поведение пользователей, которые уже перешли на новые тарифы Smart и Ultra, и на основе этого предложить другим пользователям подходящий тариф

В эту ячейку будем импортировать необходимые для решения задачи библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

Откроем и изучим файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

Таблица содежит данные о тарифе и о том, как им пользуются (звонки, минуты, сообщения, интернет).

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Хотя по условию предобработка данных уже выполнена, на всякий случай проверим, все ли данные на месте

In [5]:
df.isna().count()

calls       3214
minutes     3214
messages    3214
mb_used     3214
is_ultra    3214
dtype: int64

Разделим данные на признаки и целевой признак. Наша задача - изучить поведение пользователей, уже перешедших на новые тарифы. К признакам отнесем все, что связано с трафиком - расход минут, интернета, звонки, сообщения. Целевым признаком, который нам нужно предсказать, будет тариф. 

Мы решаем задачу классификации. Тарифы уже разделены на два типа, поэтому дополнительных изменений в таблицу не нужно.

In [6]:
features = df.drop(columns='is_ultra', axis=1)
target = df['is_ultra']

In [7]:
features.shape

(3214, 4)

In [8]:
target.shape

(3214,)

In [9]:
target.value_counts(normalize=1)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

## Разбейте данные на выборки

Для повышения качества оценки выделим проверочную выборку. В нашем условии нет спрятанной тестовой выборки, поэтому нужно разбить данные на три части: обучающую, тестовую и валидационную. Разобьем данные в соотношении 3:1:1

Для начала выделим обущающую выборку. Для этого в тестовую часть уберем 40% данных

In [10]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)

In [11]:
features_train = df_train.drop(columns='is_ultra', axis=1)
target_train = df_train['is_ultra']

In [12]:
features_train.shape

(1928, 4)

In [13]:
target_train.shape

(1928,)

In [14]:
features_valid = df_valid.drop(columns='is_ultra', axis=1)
target_valid = df_valid['is_ultra']

In [15]:
features_valid.shape

(1286, 4)

In [16]:
target_valid.shape

(1286,)

Затем поделим пополам данные, на которых мы будем проверять модель - валидационную и тестовую выборки. Таким образом, на обучающую часть у нас приходится теперь 60% данных, на валидационную - 20%, тестовую - 20%.

In [17]:
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

In [18]:
features_valid = df_valid.drop(columns='is_ultra', axis=1)
target_valid = df_valid['is_ultra']

In [19]:
features_test = df_test.drop(columns='is_ultra', axis=1)
target_test = df_test['is_ultra']

In [20]:
features_valid.shape

(643, 4)

In [21]:
target_valid.shape

(643,)

In [22]:
features_test.shape

(643, 4)

In [23]:
target_test.shape

(643,)

## Исследуйте модели

Начнем с модели решающего дерева. В теории эта модель имеет самое низкое качество предсказаний, но достаточно высокую скорость работы. Тем не менее, постараемся достичь на ней точности не менее 0.75. Так как мы не знаем, какой должен быть оптимальный параметр глубины дерева, то переберем значения от 1 до 4 включительно. Добавим еще один гиперпараметр - минимальное количество объектов в листе, который не позволит модели создавать лист дерева, в котором будет слишком мало объектов обучающей выборки. 

In [24]:
best_tree_model = None
best_tree_result = 0

for depth in range(1,5):
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_leaf=2)
    tree_model.fit(features_train, target_train)
    tree_model.predict(features_train)
    tree_result = tree_model.score(features_valid, target_valid)
    if tree_result > best_tree_result:
        tree_best_model = tree_model
        best_tree_result = tree_result

Для глубины дерева от 1 до 4 мы получили наиболее точное значение 0.785. Мы решили перебрать значения до 4, так как при меньшем значении есть риск недообчуения модели, выше - переобучения.
Можно проверить модель на более широком диапазоне (например, до 10), но точность от этого не сильно меняется.

In [25]:
best_tree_result

0.7853810264385692

Далее построим модель случайный лес. В теории она должна дать нам самое высокое качество при более низкой скорости работы.

In [26]:
%%time

best_forest_model = None
best_forest_result = 0

for est in range(1,31):
    forest_model = RandomForestClassifier(random_state=12345, n_estimators=est)
    forest_model.fit(features_train, target_train)
    forest_result = forest_model.score(features_valid, target_valid)
    if forest_result > best_forest_result:
        best_forest_model = forest_model
        best_forest_result = forest_result

CPU times: user 1.9 s, sys: 5.12 ms, total: 1.9 s
Wall time: 1.91 s


При установленных гиперпараметрах точность модели получилась 0.7947, что выше дерева решений. Как и в случае дерева решений, мы перебираем разное количество деревьев, которое будет обучать модель. Мы остановились на диапазоне от 1 до 30 включительно. При более высоких значениях скорость работы модели уменьшается, а качество меняется не так существенно.

In [27]:
best_forest_result

0.7947122861586314

<div class="alert alert-info">
прикольная штука)
</div>
</div>

Последняя модель, которую мы рассмотрим - логистическая регрессия. Она содержит небольшое количество параметров, поэтому ей не грозит переобучение. Тем не менее, качество этой модели на исследуемых данных оказалось самым низким. Можно установить различное количество итераций - 100, 1000, 10000, но результат от этого не меняется в лучшую сторону.

In [28]:
log_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=10000)
log_model.fit(features_train, target_train)
log_model.predict(features_train)
log_model.score(features_valid, target_valid)

0.7107309486780715

## Проверьте модель на тестовой выборке

Проверим рассмотренные модели на тестовой выборке. Лучший результат показывает модель случайного леса. Близко к ней - решающее дерево. Модель логистической регрессии - единственная, на которой не удалось достичь требуемой точность 0.75.

In [30]:
test_forest_predictions = best_forest_model.predict(features_test)
best_forest_model.score(features_test, target_test)

0.7807153965785381

## (бонус) Проверьте модели на адекватность

В задании написано, нестрашно, если этот раздел не получится, но бонусом все же попробуем проверить модели на адекватность) Посмотрим какую долю ответов дает каждая модель.

In [32]:
def all_models_share(features_train, target_train, features_valid, target_valid):
    tree_model = DecisionTreeClassifier(random_state=12345)
    tree_model.fit(features_train, target_train)
    tree_share = pd.Series(tree_model.predict(features_valid)).value_counts(normalize=1)
    
    forest_model = RandomForestClassifier(random_state=12345, n_estimators=30)
    forest_model.fit(features_train, target_train)
    forest_share = pd.Series(forest_model.predict(features_valid)).value_counts(normalize=1)
    
    log_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=10000)
    log_model.fit(features_train, target_train)
    log_share = pd.Series(log_model.predict(features_valid)).value_counts(normalize=1)
    
    print("Доля ответов, дерево решений:\n", tree_share, 
          "Доля ответов, случайный лес:\n", forest_share, 
          "Доля ответов, логистическая регрессия:\n", log_share)

Ранее логистическая модель оказалась самой неточной. Мы также видим, что она практически всегда выдает одинаковый результат. У двух других моделей похожее распределение ответов.

In [33]:
all_models_share(features_train, target_train, features_valid, target_valid)

Доля ответов, дерево решений:
 0    0.712286
1    0.287714
dtype: float64 Доля ответов, случайный лес:
 0    0.788491
1    0.211509
dtype: float64 Доля ответов, логистическая регрессия:
 0    0.973561
1    0.026439
dtype: float64


Сравним качество моделей с константной моделью. За константу возьмем ответ 0, так как он чаще всего встречается. 

In [34]:
target_predict_constant = pd.Series([0]*len(target_valid))
target_predict_constant.shape

(643,)

Точность константной модели существенно ниже моделей решающего дерева и случайного леса и сопоставима с логистической регрессией.

In [35]:
accuracy_score_constant = accuracy_score(target_valid, target_predict_constant)
accuracy_score_constant

0.7060653188180405

In [36]:
X = target_test
y = pd.Series([0]*len(target_test))
dummy_clf = DummyClassifier(strategy='most_frequent')
dummy_clf.fit(X, y)

DummyClassifier(strategy='most_frequent')

In [37]:
dummy_clf.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [38]:
dummy_clf.score(X, y)

1.0

## Вывод

В текущем проекте нам нужно было проанализировать данные о поведении клиентов, уже перешедших на новые тарифы, и выбрать модель, которая выбрала бы подходящий тариф для остальных клиентов. Доля правильных ответов в модели должна быть не ниже 0.75.

Мы разделили исходные данные на признаки, к которым отнесли все, что относится к пользованию тарифом - количество звонков, сообщений, расход минут и интернет трафика. Целевым признаком установили тип тарифа.

Мы также разделили данные на обучающую, валидационную и тестовую выборку в соотношении 3:1:1.

На основе этого были построены три модели.

В модели решающего дерева мы перебрали решения для различной глубины дерева - от 1 до 4. Такое значение было выбрано с целью избежать недообучения или переобучения модели. Наилучший результат оказался - 0.785.

Для модели случайного леса оптимальным гиперпараметром оказался диапазон деревьев от 1 до 30. Это позволило достичь более менее максимального качества модели при высокой скорости ее работы - 0.7947. 

Модель логистической регрессии оказалась самой слабой с результатом 0.71. Возможно, это связано с небольшим количеством параметров, которое использует модель.

Результаты тестовой выборки моделей решающего дерева и случайного леса оказались близки к валидационным, что может говорить о том, что модели не переучились или недоучились. 

Мы также проверили модели на адекватность. Проверка подтвердила неточность работы логистической модели и снова показала близость результатов двух остальных моделей, которые также оказались выше результата константной модели.